```
Langchain 을 활용한 모델 사용,비용 모니터링 및 캐싱 전략
    GPT-4o-mini  GPT-3.5-Turbo 비용이 60% 저렴
    LangChain V0.3x 부터 openAi가 별도 패키지로 분리 필요 패키지를 설치 langchain-openai 필요 
    토큰사용량 추적, 캐싱을 위한 langchain-community도 별도설치
    환경변수 변수 관리 패키지 python-dotevn
```


In [2]:
%pip install langchain-openai langchain-community python-dotenv openai

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 73.1 MB/s eta 0:00:00

   ------------------------- -------------- 5/8 [pydantic-settings]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ----------------------------------- ---- 7/8 [langchain-community]
   ------------

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4o-mini',temperature=0.7)  # dotenv 를 하면 자동으로 OPENAI_API_KEY 참조
prompt = 'LangChain에 대해 한 문장으로 설명해줘'
result = llm.invoke(prompt)
result.content

'LangChain은 언어 모델을 활용하여 다양한 애플리케이션을 구축할 수 있도록 지원하는 프레임워크입니다.'

In [6]:
#사용량
result.usage_metadata

{'input_tokens': 18,
 'output_tokens': 28,
 'total_tokens': 46,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [7]:
# 콜벡함수를 통한 누적 토큰 추적(get_openai_callback)
from langchain_community.callbacks import get_openai_callback
with get_openai_callback() as cb:
    # 첫번째 호출
    res1 = llm.invoke('서울의 오늘 날씨는 어떤지 알려줘?')
    print('응답1', res1.content[:10],'...')
    # 두번째 호출
    res2 = llm.invoke('파이썬으로 랭체인 사용법을 알려줘')
    print('응답2', res2.content[:10],'...')

# 누적 토큰 사용량 출력  콜백 cb에는 블록 내 전체 토큰 사용량이 누적
# 총 토큰수
print('총 토큰수:', cb.total_tokens)
# 프롬프트 토큰수
print('프롬프트 토큰수:', cb.prompt_tokens)
# 응답 토큰수
print('응답 토큰수:', cb.completion_tokens)
# 비용 계산
print('비용(USD):', cb.total_cost)


응답1 죄송하지만, 실시간 ...
응답2 랭체인(LangCh ...
총 토큰수: 856
프롬프트 토큰수: 39
응답 토큰수: 817
비용(USD): 0.0004960499999999999


In [ ]:
# LangChain의 LLM 응답캐싱 (InMemory Cache, SQLiteCache)
# 동일한 질문은 저장해 뒀다가 응답에 사용